# Gradient Boosting for Customer Churn Prediction

This notebook evaluates Gradient Boosting as a more expressive model for customer
churn prediction and compares its performance to the logistic regression baseline.


**1. Imports**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, classification_report

**2. Load data & preprocessing**

In [2]:
df = pd.read_csv("../data/raw_data.csv")
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

X = df.drop("Churn", axis=1)
y = df["Churn"].map({"Yes": 1, "No": 0})

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

num_features = ["tenure", "MonthlyCharges", "TotalCharges"]
cat_features = [col for col in X.columns if col not in num_features]

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ]
)

## Data Preparation

The same preprocessing pipeline as in previous notebooks is reused to ensure
a fair and consistent comparison across models.

**3. Gradient Boosting model**

In [3]:
gb_model = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", GradientBoostingClassifier(random_state=42))
    ]
)

gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_test)
y_prob_gb = gb_model.predict_proba(X_test)[:, 1]

roc_auc_gb = roc_auc_score(y_test, y_prob_gb)
roc_auc_gb  

0.8457503422976569

**4. Evaluation**

In [4]:
print(classification_report(y_test, y_pred_gb))

              precision    recall  f1-score   support

           0       0.84      0.91      0.87      1035
           1       0.67      0.51      0.58       374

    accuracy                           0.80      1409
   macro avg       0.75      0.71      0.73      1409
weighted avg       0.79      0.80      0.79      1409



### Evaluation Results

The Gradient Boosting model demonstrates improved predictive performance compared
to the linear baseline, indicating the presence of non-linear patterns and
feature interactions in customer churn behavior.

**5. Comparison with Baseline**

## Comparison with Logistic Regression

Compared to the Logistic Regression baseline, Gradient Boosting achieves higher
predictive performance, particularly in terms of churn recall and overall
discriminative ability (ROC-AUC).

This improvement comes at the cost of reduced interpretability, highlighting
the trade-off between model complexity and transparency.

While Gradient Boosting achieves a higher ROC-AUC compared to the baseline,
its recall for churned customers at the default probability threshold (0.5)
is slightly lower. This indicates that the model is more conservative in
assigning the churn class, highlighting the importance of threshold selection
in churn prediction tasks.

**6. Summary**

### Summary

This experiment confirms that customer churn is influenced by non-linear
relationships that cannot be fully captured by linear models.

Gradient Boosting provides a meaningful performance improvement and represents
a strong candidate for deployment or further explainability analysis.